In [11]:
from glob import glob
import os

input_path = sorted(glob(os.path.join("B:\MastherThesisDaten\DigiStrucMed_Braesen\ABC" , "*.svs")))

path, file = os.path.split(input_path[0])
filename, ext = os.path.splitext(file)
try:
    # Haematoxylin and eosin OR IHC
    case, stain_code = filename.split('~')[0].split('.')
except:
    # Haemalun
    case, _, stain_code = filename.split('~')[0].split('.')

print(f"case: {case}, stain_code: {stain_code}")

case: RCC-TA-011, stain: 023


In [1]:
import monai
from monai import transforms
from monai.data import (DataLoader, WSIReader, create_test_image_2d,
                        decollate_batch, list_data_collate)
from monai.data.wsi_reader import TiffFileWSIReader, WSIReader
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.transforms import (Activations, AsChannelFirst, AsDiscrete, Compose,
                              Crop, CropForegroundd, EnsureChannelFirstd,
                              EnsureTyped, LoadImage, LoadImaged,
                              MaskIntensityd, RandCropByPosNegLabeld,
                              RandRotate90d, Resized, ScaleIntensityd)
from monai.visualize import plot_2d_or_3d_image

Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory


In [1]:
from glob import glob
import os

base_path = "/mnt/b/" # "/media/sean/USB/"
scan_path = "/home/sklein/Thesis/Scans-QuPathProjekt-RCC(1Case)-10062021/"
maks_path = "/home/sklein/Thesis/DigiStrudMed_sklein/AnnotationMasks_old/Tissue_Masks/"
images = sorted(glob(os.path.join(scan_path , "*.svs")))
#segs = sorted(glob(os.path.join(maks_path, "*.ome.tif"))) # *labels.ome.tif
masks = sorted(glob(os.path.join(maks_path, "*.ome.tif"))) # *Tumor.ome.tif

train_files = [
    {
        "img": img, 
        "mask":mask, 
        # "seg": seg,
        } 
    for img, mask, #seg 
    in zip(
        images[:20],
        masks[:20],
        #segs[:20],
        )]
# val_files = [{"img": img, "mask":mask, "seg": seg} for img, mask, seg in zip(images[-20:], masks[-20:], segs[-20:])]
print(train_files)

[]


In [4]:
smallest_image = min(images, key=lambda x: x.shape[:2])
smallest_height, smallest_width = smallest_image.shape[:2]
resize_transform = Resized(keys=["image", "mask"], spatial_size=(smallest_height, smallest_width))

wsi_transforms = Compose(
    [
        LoadImaged(keys=["img", "mask", ], reader=TiffFileWSIReader , level=3),
        EnsureChannelFirstd(keys=["img", "mask", ]),
        # select pixels with value > 0 from "mask" as foreground (default function)
        CropForegroundd(keys=["image","mask"], source_key="mask"),

        # replace background with black from mask image
        MaskIntensityd(keys=["image", "mask"],select_fn= lambda x: x != 0, mask_key="mask")
        
        transforms.ToTensor(), # needed?
        # transforms.CuCIM() # <-- this could be useful for image registration
        
        transforms.Lambda(
            lambda image: Resized(keys=["image", "mask"], 
            spatial_size=(
                min(image, key=lambda x: x.shape[:2])
                .shape[:2])(image))
                ),
        # Scales intensity to [0, 1] (default function)
        ScaleIntensityd(keys=["img", "seg"]),
    ]
)


# get the min shape of a set of tensors
# min_shape = min([img.shape for img in images])
# print(f"min_shape = {min_shape}")

SyntaxError: invalid syntax (3554330637.py, line 12)

In [5]:
import numpy as np
from monai.transforms import CropForeground, MaskIntensityd

# transform = LoadImage(reader=WSIReader, backend='openslide', level=3)
# image, _ = transform(images[0])
# print(f"LoadImage : image shape = {image.shape}")

# transformD = LoadImaged(keys=["image"], reader=WSIReader, backend='openslide', level=3)
# image = transformD({"image": images[0]})["image"]
# print(f"LoadImageD: image shape = {image.shape}")

image = np.array(
    [[[0, 0, 0, 0, 0],
      [0, 1, 2, 1, 0],
      [0, 1, 3, 2, 0],
      [0, 1, 2, 1, 0],
      [0, 0, 0, 0, 1]]])  # 1x5x5, single channel 5x5 image


mask = np.array(
    [[[0, 0, 0, 0, 0],
      [0, 1, 2, 1, 0],
      [0, 1, 3, 2, 0],
      [0, 1, 2, 1, 0],
      [0, 0, 0, 0, 0]]])

def threshold_at_one(x):
    # threshold at 1
    return x > 0

removeBack = MaskIntensityd(keys=["image", "mask"],select_fn= lambda x: x != 0, mask_key="mask")

cropper = CropForeground(select_fn=threshold_at_one, margin=0)
print(removeBack(image))
# [[[2, 1],
#   [3, 2],
#   [2, 1]]]

# mask == 0

ValueError: dictionary update sequence element #0 has length 5; 2 is required

In [10]:
from cucim import CuImage
import pprint as pp
from matplotlib import pyplot as plt
import numpy as np
import os

def visualize(image, downsample=1):
    dpi = 80.0 * downsample
    height, width, _ = image.shape
    plt.figure(figsize=(width / dpi, height / dpi))
    plt.axis('off')
    plt.imshow(image)
    return True
plt.close('all')

img = CuImage(masks[0])
pp.pprint(img.metadata)#.get('cucim'))
curr_img = img.read_region(level=3)
visualize(np.asarray(curr_img), 1) 
#import cupy as cp

TIFFReadDirectory: Warning, Incorrect count for "ColorMap"; tag ignored.
[Warning] Loading image('/home/sklein/Thesis/DigiStrudMed_sklein/AnnotationMasks_old/Tissue_Masks/RCC-TA-011.001.023~M-Tissue.ome.tif') with a slow-path. The pixel format of the loaded image would be RGBA (4 channels) instead of RGB!


{'cucim': {'associated_images': [],
           'channel_names': ['R', 'G', 'B', 'A'],
           'coord_sys': 'LPS',
           'dims': 'YXC',
           'direction': [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]],
           'dtype': {'bits': 8, 'code': 1, 'lanes': 1},
           'ndim': 3,
           'origin': [0.0, 0.0, 0.0],
           'path': '/home/sklein/Thesis/DigiStrudMed_sklein/AnnotationMasks_old/Tissue_Masks/RCC-TA-011.001.023~M-Tissue.ome.tif',
           'resolutions': {'level_count': 1,
                           'level_dimensions': [[100105, 98661]],
                           'level_downsamples': [1.0],
                           'level_tile_sizes': [[512, 512]]},
           'shape': [98661, 100105, 4],
           'spacing': [0.1729000061750412, 0.1729000061750412, 1.0],
           'spacing_units': ['micrometer', 'micrometer', 'color'],
           'typestr': '|u1'},
 'tiff': {'model': '',
          'resolution_unit': 'centimeter',
          'software': 'OME Bio-Fo

KeyboardInterrupt: 

Error in callback <function _draw_all_if_interactive at 0x7f848648f5e0> (for post_execute):


ValueError: Image size of 125131x123326 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 125131x123326 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 125131x123326 with 1 Axes>

In [ ]:
import numpy as np

bla = np.asarray(curr_img)
bla.shape

bla2 = np.asarray(img)
print(bla2)

print(img)
print(curr_img)

In [ ]:
transform = LoadImage(reader=WSIReader, backend='openslide', level=0)
image, _ = transform(train_files[0]["seg"])

In [ ]:
check_ds = monai.data.Dataset(data=train_files, transform=transforms) # type: ignore
check_ds[0]["img"].shape